In [107]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math


## Prepare data

In [109]:
gender_df= pd.read_csv('../input/titanic/gender_submission.csv')
# test_df = pd.read_csv('../input/titanic/test.csv')
train_df = pd.read_csv('../input/titanic/train.csv')
print(train_df.isnull().sum(axis = 0))


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [111]:
from sklearn.impute import SimpleImputer
si = SimpleImputer(strategy="most_frequent")
inputerResult = si.fit_transform(train_df[['Embarked']])

train_df['Embarked'] = inputerResult

si = SimpleImputer(strategy="mean")
inputerResult = si.fit_transform(train_df[['Age']])
train_df['Age'] = inputerResult
train_df["Age"] = train_df['Age'].map(lambda x: '{:.2f}'.format(float(x)))


In [113]:
print(train_df.isnull().sum(axis = 0))


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64


## Check dataset on null cells

In [115]:
# qwe = train_df.groupby(["Embarked"]).agg({'Survived': ['count', 'sum']}).reset_index()

# v = pd.DataFrame.from_records(qwe.Survived) 
# v = pd.concat([qwe.drop('Survived', 1), v], axis=1)
# v['qwe'] = v['sum'] * 100 / v['count'] 
# print(v)
# ax = sns.barplot(x="count", y="qwe", data=v)

In [117]:
# train_df['dead'] =  train_df.apply(lambda row: (row['Survived'] - 1)**2, axis=1)
# qwe = train_df[train_df['Sex']=='female']

# qwe = qwe.groupby(["Age"]).agg({'dead': 'sum', 'Survived': 'sum'}).reset_index()

# qwe["Age"] = qwe['Age'].map(lambda x: '{:.2f}'.format(float(x)))
# qwe["Age"] = qwe["Age"].astype(float)

# qwe['surv'] =  qwe.apply(lambda row: (row['Survived'] * 100) / (row['dead'] + row['Survived']), axis=1)

# print(qwe.head())
# fig, ax = plt.subplots(figsize=(50,10))
# print(qwe["Age"].dtype)
# ax = sns.barplot(x="Age", y='surv', data=qwe, ax=ax)

In [119]:
def binAge(row):
    if row['Sex'] == 'male':
        if float(row['Age']) < 16:
            return 'alive'
        else:
            return 'dead'
    else:
        return 'itsFemale'

train_df['bin_male_age'] = train_df.apply(binAge, axis=1)

In [121]:
def withNanny(row):
    if float(row['Parch']) == 0 and float(row['Age']) < 18:
        return 'with'
    elif float(row['Parch']) != 0 and float(row['Age']) < 18:
        return 'without'
    else:
        return 'adult'
    
train_df['withNanny'] = train_df.apply(withNanny, axis=1)

In [123]:
print("Only {}% of data have Cabin number".format(100 - train_df.Cabin.isnull().sum(axis = 0)*100/len(train_df.Cabin)))

Only 22.895622895622893% of data have Cabin number


In [125]:
features = [ 'Age', 'SibSp', 'Parch', 'Fare']
cat_features = ['Embarked', 'Sex','Pclass' , 'withNanny']
todo = ['Cabin', 'Ticket']

In [127]:
base_model = train_df[features]
cat_df = train_df[cat_features]

In [129]:
cat_df.index

RangeIndex(start=0, stop=418, step=1)

In [131]:
base_model

,Age,SibSp,Parch,Fare
0,34.50000,0,0,7.8292
1,47.00000,1,0,7.0000
2,62.00000,0,0,9.6875
3,27.00000,0,0,8.6625
4,22.00000,1,1,12.2875
...,...,...,...,...
413,30.27259,0,0,8.0500
414,39.00000,0,0,108.9000
415,38.50000,0,0,7.2500
416,30.27259,0,0,8.0500


In [133]:
data = pd.get_dummies(cat_df, columns=['Embarked'], prefix="Embarked", drop_first=True)
data = pd.get_dummies(data, columns=['Sex'], prefix="Sex", drop_first=True)
data = pd.get_dummies(data, columns=['Pclass'], prefix="Pclass", drop_first=True)
data = pd.get_dummies(data, columns=['withNanny'], prefix="withNanny", drop_first=True)

In [135]:
data

,Embarked_Q,Embarked_S,Sex_male,Pclass_2,Pclass_3,withNanny_with,withNanny_without
0,1,0,1,0,1,0,0
1,0,1,0,0,1,0,0
2,1,0,1,1,0,0,0
3,0,1,1,0,1,0,0
4,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...
413,0,1,1,0,1,0,0
414,0,0,0,0,0,0,0
415,0,1,1,0,1,0,0
416,0,1,1,0,1,0,0


In [137]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, train_df.Survived, test_size = 0.25, random_state = 0)

ValueError: Found input variables with inconsistent numbers of samples: [418, 891]

In [140]:
# Fitting SVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel='rbf', random_state = 123)
classifier.fit(X_train, y_train)

SVC(random_state=123)

In [142]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [143]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8161434977578476

In [144]:
test_df = pd.read_csv('../input/titanic/test.csv')
print(test_df.isnull().sum(axis = 0))


PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [145]:
si = SimpleImputer(strategy="mean")
inputerResult = si.fit_transform(test_df[['Age']])
test_df['Age'] = inputerResult

In [146]:
test_df['bin_male_age'] = test_df.apply(binAge, axis=1)
test_df['withNanny'] = test_df.apply(withNanny, axis=1)

In [147]:
test_df.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,bin_male_age,withNanny
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,dead,adult
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,itsFemale,adult
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,dead,adult
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,dead,adult
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,itsFemale,adult


In [148]:
base_model = test_df[features]
cat_df = test_df[cat_features]

data = pd.get_dummies(cat_df, columns=['Embarked'], prefix="Embarked", drop_first=True)
data = pd.get_dummies(data, columns=['Sex'], prefix="Sex", drop_first=True)
data = pd.get_dummies(data, columns=['Pclass'], prefix="Pclass", drop_first=True)
data = pd.get_dummies(data, columns=['withNanny'], prefix="withNanny", drop_first=True)

In [149]:
 y_pred = classifier.predict(data)

In [150]:
y_pred

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [151]:
resultDF = pd.DataFrame(index=test_df.index)

In [152]:
resultDF['PassengerId'] = pd.Series(test_df['PassengerId'], index=resultDF.index, dtype=int)
resultDF['Survived'] = pd.Series(y_pred, index=resultDF.index, dtype=int )

In [153]:
for index, row in test_df.iterrows():
    resultDF['PassengerId'][index] = row.PassengerId

In [154]:
resultDF

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [155]:
resultDF.to_csv('out.csv', index=False)